In [ ]:
# Install required packages
!pip install scikit-learn==1.2.2
!pip install tensorflow-data-validation
!pip install eli5
!pip install lime
!pip install tf-explain
!pip install fairlearn

In [ ]:
# Import required packages

import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
import eli5
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
)
from fairlearn.metrics import selection_rate
from eli5.sklearn import PermutationImportance
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
import lime
from lime.lime_tabular import LimeTabularExplainer
import tensorflow as tf
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load the sample maternal health risk dataset
health_risk_data = pd.read_csv("./Maternal Health Risk Data Set.csv")
print(health_risk_data.shape)
health_risk_data.head()

In [ ]:
# Split the data into training and validation sets
X = health_risk_data.drop('RiskLevel', axis=1)  # Features
y = health_risk_data['RiskLevel']  # Labels


# Split with a 70-30 train-eval ratio (you can adjust this ratio)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# As the data originally doesn't contain any missing values or anomalies, we are introducing those data issues to show you how to treat them
for col in X_train.columns:
  X_train.loc[X_train.sample(frac=0.05).index, col] = np.nan

for col in X_val.columns:
  X_val.loc[X_val.sample(frac=0.05).index, col] = np.nan

# Introduce anomaly values in Age feature
X_val.loc[X_val.sample(frac=0.01).index, "Age"] *= 20


In [ ]:
# Step 1: Generate Descriptive Statistics for Train and Val dataset
train_stats = tfdv.generate_statistics_from_dataframe(X_train)
val_stats = tfdv.generate_statistics_from_dataframe(X_val)

In [ ]:
tfdv.visualize_statistics(train_stats)

In [ ]:
tfdv.visualize_statistics(val_stats)

In [ ]:
# Correlation heatmap for numeric columns. We are not including the Target variable here. This will help us understand if there is any multicollinearity

correlation_matrix = X_train.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

In the above correlation map you can see that SystolicBP is highly correlated with DiastolicBP. Looking at this from a clinical lens, we understand that blood pressure is measured using two values—systolic and diastolic. SystolicBP represents the maximum arterial pressure during a cardiac cycle when the heart is contracting (systole), while DiastolicBP represents the minimum arterial pressure when the heart is at rest (diastole). The two values are obtained during the same blood pressure measurement, hence the correlation.

As you can see in the above two desctiptive stats from the train and validation dataset, that there are missing values in each of the columns. In the next section, we will learn to impute data in these missing positions.

In [ ]:
# In our dataset, all the feature values are numeric, hence we will use the same imputation method across all feature.
# If your dataset has a mix of categorical, text, and numerical values, then you will need to use different methods for data imputations.
# In our dataset as we used a random method to introduce missing values, there is no pattern behind missing values.
# But in other cases, there might be a reason why certain values might be missing, and in those cases you will need to treat them differently.


In [ ]:
# Impute missing values in numeric columns of training data with mean
numeric_imputer = SimpleImputer(strategy='mean')
X_train[X_train.columns] = numeric_imputer.fit_transform(X_train)

# Imputing values in validation dataset
# Imputing data in validation is different than train data. 
# In train you do fit_transform, which means you are deriving the value of the mean from that data and imputing in the same data.
# Whereas, you shouldn't be deriving the imputation metric from the validation or test data, hence we use .transform for val and test dataset.

X_val[X_val.columns] = numeric_imputer.transform(X_val)


## Detecting outliers or anomalies

In [ ]:
sns.boxplot(X_train)

In [ ]:
# Method 1: Outlier detection using Z-Score
outliers_zscore = {}
threshold = 3  # Adjust the threshold as needed
for column in X_train.columns:
    z_scores = np.abs(stats.zscore(X_train[column]))
    outliers_zscore[column] = X_train[column][z_scores > threshold]

print("Outliers detected using Z-Score for X_train:")
for column, outliers in outliers_zscore.items():
    print(f"{column}: {list(outliers)}")


In [ ]:
# Method 2: Outlier detection using IQR (Interquartile Range)
outliers_iqr = {}
for column in X_train.columns:
    Q1 = X_train[column].quantile(0.25)
    Q3 = X_train[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_iqr[column] = X_train[column][(X_train[column] < lower_bound) | (X_train[column] > upper_bound)]

print("Outliers detected using IQR:")
for column, outliers in outliers_iqr.items():
    print(f"{column}: {list(outliers)}")

In [ ]:
# Step 2: Infer Schema for Train and Eval dataset
train_schema = tfdv.infer_schema(train_stats)
eval_schema = tfdv.infer_schema(val_stats)

In [ ]:
dom = schema_pb2.FloatDomain(name="Age", min=1, max=90) # creating domain buffer
tfdv.set_domain(schema=train_schema,feature_path="Age",domain=dom) # setting domain
tfdv.display_schema(train_schema)

# Similarly you can edit the schema and specify details about how the ideal data would look like

In [ ]:
# Step 3: Calculate Anomalies in Eval data compared to the Train data
# In this cell you will see that the anomalies we introduced with the Age Feature is detected.

anomalies = tfdv.validate_statistics(val_stats, train_schema)
tfdv.display_anomalies(anomalies)

# In the cell below you can see that the values in the Age field is not the same as we found in the train schema

In [ ]:
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=val_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

In [ ]:
# For our use-case we will be using skew_comparator and it is appropriate for our use-case of comparing the data distribution of train and eval dataset.
# In this cell we are using Jensen Shannon Divergence Metric to evaluate the skew for "Age" feature in the dataset. 
# The threshhold is determined using statistical method that is not covered in this course.
# The recommendation is for the learners to replicate similar tests for other features, and document their observations about the data they are working with.

tfdv.get_feature(train_schema, 'Age').skew_comparator.jensen_shannon_divergence.threshold = 0.01
skew_anomalies = tfdv.validate_statistics(
  statistics=train_stats,
  schema=train_schema,
  serving_statistics=val_stats)
tfdv.display_anomalies(skew_anomalies)


# Model Validation
Now lets build a simple classifier model for the above dataset, and then we will see a few methods for model validation.

In [ ]:
# # Select a classification algorithm (Random Forest in this example)
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the testing data
y_pred = model.predict(X_val)

# Calculate accuracy for Holdout Validation
accuracy_holdout = accuracy_score(y_val, y_pred)

# Calculate other evaluation metrics
confusion_matrix_holdout = confusion_matrix(y_val, y_pred)
classification_report_holdout = classification_report(y_val, y_pred)

# Print the results for Holdout Validation
print("Holdout Validation Results:")
print("Accuracy:", accuracy_holdout)
print("Confusion Matrix:")
print(confusion_matrix_holdout)
print("Classification Report:")
print(classification_report_holdout)



In [ ]:
# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
for train_index, test_index in kf.split(X):
    X_train_kf, X_val_kf = X.iloc[train_index], X.iloc[test_index]
    y_train_kf, y_val_kf = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train_kf, y_train_kf)
    y_pred_kf = model.predict(X_val_kf)
    accuracy_kf = accuracy_score(y_val_kf, y_pred_kf)
    accuracies.append(accuracy_kf)

# Print K-Fold Cross Validation Results
print("K-Fold Cross Validation Results (Accuracies):")
for i, accuracy in enumerate(accuracies):
    print(f"Fold {i + 1}: {accuracy:.4f}")

In [ ]:
# Bootstrap Validation (using Random Forest)
n_bootstrap_samples = 100
bootstrap_accuracies = []
for _ in range(n_bootstrap_samples):
    indices = np.random.choice(X_train.index, size=len(X_train), replace=True)
    X_bootstrap = X_train.loc[indices]
    y_bootstrap = y_train.loc[indices]
    model.fit(X_bootstrap, y_bootstrap)
    y_pred_bootstrap = model.predict(X_val)
    accuracy_bootstrap = accuracy_score(y_val, y_pred_bootstrap)
    bootstrap_accuracies.append(accuracy_bootstrap)

# Print Bootstrap Validation Results
print("Bootstrap Validation Results (Accuracies):")
for i, accuracy in enumerate(bootstrap_accuracies):
    print(f"Sample {i + 1}: {accuracy:.4f}")

If your target variable has three classes, the ROC curve is not directly applicable because ROC curves and AUC (Area Under the Curve) are typically used for binary classification problems. However, you can calculate and visualize the ROC and AUC for each class separately in a one-vs-all fashion (also known as one-vs-rest) or use another suitable metric like a multiclass version of AUC, which is often referred to as the "micro-average" AUC.

In [ ]:
# Predict probabilities for each class on the testing data
y_probs = model.predict_proba(X_val)

# Define a mapping from text class labels to integer labels
class_mapping = {'high risk': 0, 'low risk': 1, 'mid risk': 2}

# Convert the text class labels in y_test to integer labels
y_train_int = y_train.map(class_mapping)
y_val_int = y_val.map(class_mapping)
y_pred_int = np.array([class_mapping[p] for p in y_pred])


# Calculate and plot the ROC curve and AUC for each class
n_classes = len(class_mapping)
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calculate ROC curve and AUC for each class
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_val_int == i, y_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curves for each class
plt.figure()
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multiclass Classification')
plt.legend(loc='lower right')
print(class_mapping)
plt.show()

Based on the above range of metrics, you need to understand if the model performance satisfies the business need.

# Model Explainability and Interpretability

In this next section, we will be covering Explainability and Interpretability of the model we built above.

In [ ]:
explainer = LimeTabularExplainer(X_train.to_numpy(), mode="classification")

# Define a sample prediction function
def predict_function(x):
    return model.predict_proba(x)

# Select a sample instance for explanation
sample_instance = X_val.iloc[1]

# Explain the prediction for the sample instance
explanation = explainer.explain_instance(sample_instance, predict_function, num_features=5)
# Print the explanation
explanation.show_in_notebook()

In the context of multiclass classification (more than two classes), "NOT 1" indicates that the explanation is for a class other than Class 1. It helps in understanding why a particular instance is classified as a specific class when it's not the first class in the class labels.

* LIME fits simple linear models locally around a prediction to approximate the behavior of complex models like Random Forests. This provides local explainability.

* The LIME output shows the top features that influenced a particular prediction, along with the weight of their contribution.

* Positive weights increased the prediction probability, negative weights decreased it. Higher magnitude means more important.

For this sample, we see maternal age and previous C-sections were the main drivers of the risk prediction. LIME provides instance-level explanations for model predictions. It helps explain why the model made a specific decision.

In [ ]:
eli5.show_prediction(model, X_val.iloc[1],
                    feature_names=list(X_train.columns),
                    show_feature_values=True)

In [ ]:
eli5.show_prediction(model, X_val.iloc[2],
                    feature_names=list(X_train.columns),
                    show_feature_values=True)

In [ ]:
# Initialize the Permutation Importance explainer
perm = PermutationImportance(model, random_state=42)

# Fit the explainer to your test data (e.g., X_test, y_test)
perm.fit(X_val, y_val)

# Display feature importances
permutation_importance = eli5.show_weights(perm, feature_names=X_val.columns.tolist())
permutation_importance

# Bias Detection and Mitigation

In this section we will be seeing if the model we have built has any bias, and then we will try to mitigate it
As FairLearn Package doesn't support multi-class target variable evaluation, we will do a slight modification in our data. We will make our target variable binary class

In [ ]:
# We are going to map low and medium risk in one bucket and keep high risk as another bucket
binary_class_mapping = {'high risk': 1, 'low risk': 0, 'mid risk': 0}
y_binary = y.map(binary_class_mapping)

In [ ]:
# Split with a 70-30 train-eval ratio (you can adjust this ratio)
X_train_bin, X_val_bin, y_train_bin, y_val_bin = train_test_split(X, y_binary, test_size=0.3, random_state=42)

bin_model = RandomForestClassifier(n_estimators=100, random_state=42)
bin_model.fit(X_train_bin, y_train_bin)
y_pred_bin = bin_model.predict(X_val_bin)

In [ ]:
# Define age groups
age_groups = {
    "young": (0, 18),
    "adult": (19, 59),
    "senior": (60, np.inf)
}

# Calculate selection rates by age group
selection_rates = {}
for age_group, age_range in age_groups.items():
    age_mask = X_val_bin['Age'].between(age_range[0], age_range[1])
    selection_rates[age_group] = selection_rate(y_val_bin[age_mask], y_pred_bin[age_mask])

# Check for disparate impact in selection rates
disparate_impact = max(selection_rates.values()) - min(selection_rates.values())
print("Disparate Impact by Age Group:")
print(disparate_impact)

When dealing with a multiclass target column, mitigation techniques need to be adapted accordingly. One approach to mitigate bias in multiclass classification is to address individual class disparities.



The Disparate Impact value in the fairness_metrics , which is calculated using the selection_rate fairness metric, means that, on average, there is a x% difference in the selection rates between the subgroups defined by the 'Age' sensitive feature.

In the context of selection_rate, a value of x represents a disparity in the proportion of positive predictions (selected instances) between different age groups. This means that, on average, one age group is being selected at a rate x% higher (or lower) than another age group.

The specific interpretation of this disparity depends on the context of your application and fairness requirements. A value of x suggests that there is some level of imbalance or bias in the selection rates across different age groups, but whether this is considered acceptable or not depends on the specific fairness goals and the impact of such disparities on your application.

The current stable version of FairLearn 0.9.0 doesn't support multiclass classification. Hence I have tried to make this a binary classification problem to show how the bias mitigation would look like. 

In [ ]:
# Define a disparate impact remover with Demographic Parity constraint
di_remover = ExponentiatedGradient(
    estimator=bin_model,  
    constraints=DemographicParity()
)

# Train the remover on the training data
di_remover.fit(X_train_bin, y_train_bin, sensitive_features=X_train_bin['Age'])

# Predict on the validation data
y_pred_fair = di_remover.predict(X_val_bin)

# Calculate selection rates for age groups after mitigation
selection_rates_fair = {}
for age_group, age_range in age_groups.items():
    age_mask = X_val_bin['Age'].between(age_range[0], age_range[1])
    selection_rates_fair[age_group] = selection_rate(y_val_bin[age_mask], y_pred_fair[age_mask])

# Check for disparate impact in selection rates after mitigation
disparate_impact_fair = max(selection_rates_fair.values()) - min(selection_rates_fair.values())
print("Disparate Impact by Age Group (After Mitigation):")
print(disparate_impact_fair)


As you see in the above cell the Disparate Impact score has reduced after bias mitigation. Remember this is example is just to show you how you can model your problem. In reality, there are many qualitative assessments that needs to be done, and requires subject matter experts and social science researchers to help with understand what features in the data can cause bias, and what would be the right metric to evaluate your model, and to event decide if to what extend of reduction in the metric value is acceptable.

Copyright @aishgrt 2023